In [1]:
using TaylorModels, IntervalArithmetic, TaylorSeries

In [2]:
const ∫ = integrate
const Interval = IntervalArithmetic.Interval

import Base:^

^(x::Interval{Float64}, n::Int) = pow(x, n)

^ (generic function with 92 methods)

In [17]:
n = 12  # order
a, b = set_variables("a b", order=n)

2-element Array{TaylorSeries.TaylorN{Float64},1}:
  1.0 a + 𝒪(‖x‖¹³)
  1.0 b + 𝒪(‖x‖¹³)

In [4]:
function calculate_set(U::TaylorN, V::TaylorN, bounds)

    xs = []
    ys = []

    # iterate over 4 sides of initial box

    num_points = 50

    b = bounds[2].lo
    for a in linspace(bounds[1].lo, bounds[2].hi, num_points)
        push!(xs, U([a, b]))
        push!(ys, V([a, b]))
    end

    a = bounds[1].hi
    for b in linspace(bounds[2].lo, bounds[2].hi, num_points)
        push!(xs, U([a, b]))
        push!(ys, V([a, b]))
    end

    b = bounds[2].hi
    for a in linspace(bounds[2].hi, bounds[2].lo, num_points)
        push!(xs, U([a, b]))
        push!(ys, V([a, b]))
    end

    a = bounds[1].lo
    for b in linspace(bounds[2].hi, bounds[2].lo, num_points)
        push!(xs, U([a, b]))
        push!(ys, V([a, b]))
    end

    return mid.(xs), mid.(ys)

end

calculate_set (generic function with 1 method)

In [20]:
function integ(u0, v0, t_interval, bounds)
    
    u = u0
    v = v0
    u_new = u0   # so exist outside loop
    v_new = v0

    # build up Taylor series by Picard:
    for i in 1:n+1   # how many iterations are required?
        u_new = u0 + ∫(   2  * u * (1 - v) )
        v_new = v0 + ∫(       -v * (1 - u) )

        u, v = u_new, v_new
    end

    # prepare Taylor Model:
    uu = TaylorModel(n, Interval(t_interval.lo), t_interval, u, 0..0, bounds)
    vv = TaylorModel(n, Interval(t_interval.lo), t_interval, v, 0..0, bounds)

    uu_new = ∫( 2 * uu * (1 - vv), u0[0] )
    vv_new = ∫(    -vv * (1 - uu), v0[0] )
    
    uΔ = uu_new.Δ
    vΔ = vv_new.Δ
    
    # make sure the intervals contain 0:
    m = mag(uΔ)
    uΔ = -m..m
    
    m = mag(vΔ)
    vΔ = -m..m
    
    @show uΔ, vΔ

     while ! ((uu_new.Δ ⊆ uu.Δ) && (vv_new.Δ ⊆ vv.Δ))
        uΔ *= 2
        vΔ *= 2
        
        @show uΔ, vΔ
        
        uu = TaylorModel(n, Interval(t_interval.lo), t_interval, u, uΔ, bounds)
        vv = TaylorModel(n, Interval(t_interval.lo), t_interval, v, vΔ, bounds)
        
        uu_new = ∫( 2 * uu * (1 - vv), u0[0] )
        vv_new = ∫(    -vv * (1 - uu), v0[0] )
        
    end
        

    # only need to bound the integral, not actually carry out
    # the whole integral

    # contract:

    for i in 1:10
       uu, vv = uu_new, vv_new
       uu_new = ∫( 2 * uu * (1 - vv), u0[0] )
       vv_new = ∫(    -vv * (1 - uu), v0[0] )
    end
    
    @show uu_new.Δ, vv_new.Δ

    U = uu_new(t_interval.hi)
    V = vv_new(t_interval.hi)
    
    return U, V, uu_new, vv_new
end

integ (generic function with 1 method)

In [6]:
using Plots
gr()

Plots.GRBackend()

In [7]:
time_step = 0.05

h = 0.05
bounds = (-h..h) * ones(2)  # bounds on a and b

u0 = Taylor1([(1..1) + a], n)   # initial conditions as function of a, b
v0 = Taylor1([(3..3) + b], n)   

@time U, V, uu_new, vv_new = integ(u0, v0, 0..time_step, bounds);

(uΔ, vΔ) = ([-3.4631e-13, 3.4631e-13], [-2.03686e-13, 2.03686e-13])
(uΔ, vΔ) = ([-6.9262e-13, 6.9262e-13], [-4.07372e-13, 4.07372e-13])
  1.556512 seconds (3.60 M allocations: 270.070 MiB, 4.43% gc time)


In [8]:
uu_new

TaylorModels.TaylorModel{Float64,TaylorSeries.TaylorN{IntervalArithmetic.Interval{Float64}}}(12, [0, 0], [0, 0.0500001],   [1, 1] + [1, 1] a + 𝒪(‖x‖¹³) + ( [-4, -4] + [-4, -4] a + [-2, -2] b + [-2, -2] a b + 𝒪(‖x‖¹³)) t + ( [8, 8] + [5, 5] a + [8, 8] b + [-3, -3] a² + [7, 7] a b + [2, 2] b² + [-1, -1] a² b + [2, 2] a b² + 𝒪(‖x‖¹³)) t² + ( [-6.66667, -6.66666] + [9.33333, 9.33334] a + [-12.6667, -12.6666] b + [14.9999, 15.0001] a² + [0.666666, 0.666667] a b + [-7.33334, -7.33333] b² + [-1.00001, -0.999999] a³ + [12.9999, 13.0001] a² b + [-4.66667, -4.66666] a b² + [-1.33334, -1.33333] b³ + [-0.333334, -0.333333] a³ b + [2.66666, 2.66667] a² b² + [-1.33334, -1.33333] a b³ + 𝒪(‖x‖¹³)) t³ + ( [-9.33334, -9.33333] + [-48.8334, -48.8333] a + [-5.33334, -5.33333] b + [-26.5001, -26.4999] a² + [-60.5001, -60.4999] a b + [4.33333, 4.33334] b² + [12.7499, 12.7501] a³ + [-45.3334, -45.3333] a² b + [-20.6667, -20.6666] a b² + [3.66666, 3.66667] b³ + [-0.250001, -0.249999] a⁴ + [9.74999, 9.75001] a

In [ ]:
xs, ys = calculate_set(U, V, bounds)

In [9]:
plot(xs, ys, aspect_ratio=1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.78 
 
 
 0.80 
 
 
 0.82 
 
 
 0.84 
 
 
 0.86 
 
 
 2.950 
 
 
 2.975 
 
 
 3.000 
 
 
 3.025 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [10]:
u0 = Taylor1([U], n)   # initial condition as function of a, b
v0 = Taylor1([V], n)   # initial condition as function of a, b

U, V = integ(u0, v0, time_step..(2*time_step), bounds);

xs, ys = calculate_set(U, V, bounds)

(uΔ, vΔ) = ([-2.90023e-13, 2.90023e-13], [-1.62666e-13, 1.62666e-13])
(uΔ, vΔ) = ([-5.80046e-13, 5.80046e-13], [-3.25332e-13, 3.25332e-13])


([0.646264, 0.647618, 0.648972, 0.650326, 0.65168, 0.653034, 0.654388, 0.655742, 0.657095, 0.658448  …  0.643931, 0.64419, 0.644448, 0.644707, 0.644967, 0.645226, 0.645485, 0.645744, 0.646004, 0.646264], [2.888, 2.88848, 2.88897, 2.88945, 2.88994, 2.89043, 2.89091, 2.8914, 2.89188, 2.89237  …  2.90559, 2.90364, 2.90168, 2.89973, 2.89777, 2.89582, 2.89386, 2.89191, 2.88995, 2.888])

In [11]:
plot(xs, ys, aspect_ratio=1)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.64 
 
 
 0.66 
 
 
 0.68 
 
 
 0.70 
 
 
 2.900 
 
 
 2.925 
 
 
 2.950 
 
 
 2.975 
 
 
 3.000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [46]:
step = 0.1

h = 0.05
bounds = (-h..h) * ones(2)  # bounds on a and b

u0 = Taylor1([(1..1) + (1..1)*a], n)   # initial condition as function of a, b
v0 = Taylor1([(3..3) + (1..1)*b], n)   # initial condition as function of a, b

U, V = integ(u0, v0, 0..step, bounds)

xs, ys = calculate_set(U, V, bounds)


Us = [u0(0)]
Vs = [v0(0)]

for i in 1:60
    @show i
    u0 = Taylor1([U], n)   # initial condition as function of a, b
    v0 = Taylor1([V], n)   # initial condition as function of a, b

    U, V = integ(u0, v0, (i*step)..((i+1)*step), bounds);
    
    push!(Us, U)
    push!(Vs, V)
end


(uΔ, vΔ) = ([-3.20024e-09, 3.20024e-09], [-1.85023e-09, 1.85023e-09])
(uΔ, vΔ) = ([-6.40048e-09, 6.40048e-09], [-3.70046e-09, 3.70046e-09])
(uΔ, vΔ) = ([-1.2801e-08, 1.2801e-08], [-7.40092e-09, 7.40092e-09])
(uu_new.Δ, vv_new.Δ) = ([-4.76866e-09, 5.99894e-09], [-3.46317e-09, 2.8414e-09])
i = 1
(uΔ, vΔ) = ([-1.11629e-09, 1.11629e-09], [-5.74319e-10, 5.74319e-10])
(uΔ, vΔ) = ([-2.23257e-09, 2.23257e-09], [-1.14864e-09, 1.14864e-09])
(uΔ, vΔ) = ([-4.46514e-09, 4.46514e-09], [-2.29728e-09, 2.29728e-09])
(uu_new.Δ, vv_new.Δ) = ([-1.44199e-09, 1.87386e-09], [-1.07283e-09, 9.17483e-10])
i = 2
(uΔ, vΔ) = ([-1.62596e-10, 1.62596e-10], [-9.97749e-11, 9.97749e-11])
(uΔ, vΔ) = ([-3.25191e-10, 3.25191e-10], [-1.9955e-10, 1.9955e-10])
(uΔ, vΔ) = ([-6.50382e-10, 6.50382e-10], [-3.991e-10, 3.991e-10])
(uu_new.Δ, vv_new.Δ) = ([-2.43572e-10, 1.67909e-10], [-1.15575e-10, 1.57265e-10])
i = 3
(uΔ, vΔ) = ([-6.86878e-11, 6.86878e-11], [-3.8993e-11, 3.8993e-11])
(uΔ, vΔ) = ([-1.37376e-10, 1.37376e-10], [-7.79

(uu_new.Δ, vv_new.Δ) = ([-8.19995e-16, 1.66516e-16], [-5.98087e-17, 1.84196e-16])
i = 35
(uΔ, vΔ) = ([-9.96557e-16, 9.96557e-16], [-2.60719e-16, 2.60719e-16])
(uΔ, vΔ) = ([-1.99312e-15, 1.99312e-15], [-5.21438e-16, 5.21438e-16])
(uu_new.Δ, vv_new.Δ) = ([-1.24702e-15, 2.42662e-16], [-9.49871e-17, 2.70545e-16])
i = 36
(uΔ, vΔ) = ([-1.54164e-15, 1.54164e-15], [-4.54945e-16, 4.54945e-16])
(uΔ, vΔ) = ([-3.08328e-15, 3.08328e-15], [-9.09889e-16, 9.09889e-16])
(uu_new.Δ, vv_new.Δ) = ([-1.96062e-15, 8.24676e-16], [-3.60196e-16, 4.82778e-16])
i = 37
(uΔ, vΔ) = ([-3.061e-15, 3.061e-15], [-1.51876e-15, 1.51876e-15])
(uΔ, vΔ) = ([-6.122e-15, 6.122e-15], [-3.03752e-15, 3.03752e-15])
(uu_new.Δ, vv_new.Δ) = ([-3.68249e-15, 4.1183e-15], [-1.63061e-15, 1.13083e-15])
i = 38
(uΔ, vΔ) = ([-1.61515e-14, 1.61515e-14], [-7.29819e-15, 7.29819e-15])
(uΔ, vΔ) = ([-3.2303e-14, 3.2303e-14], [-1.45964e-14, 1.45964e-14])
(uu_new.Δ, vv_new.Δ) = ([-1.23181e-14, 2.18883e-14], [-7.80247e-15, 4.34774e-15])
i = 39
(uΔ, v

In [19]:
using Interact

INFO: Recompiling stale cache file /Users/dpsanders/.julia/lib/v0.6/Interact.ji for module Interact.


In [29]:
typeof(Us[1])

TaylorSeries.TaylorN{IntervalArithmetic.Interval{Float64}}

In [48]:
# Calculate sets for each step:

xsets = []
ysets = []

for i in 1:61
    xs, ys = calculate_set(Us[i], Vs[i], bounds)
    push!(xsets, xs)
    push!(ysets, ys)
end

Plot them all:

In [50]:
p = plot(xlim=(0, 5), ylim=(0, 3), leg=false, aspect_ratio=1)

for i in 1:length(xsets)
    
    plot!(xsets[i], ysets[i], linetype=:shape, alpha=0.5)

end

p

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polygon clip-path="url(#clip7802)" points="
128.353,31.9988 128.581,31.9988 128.809,31.9988 129.037,31.9988 129.264,31.9988 129.492,31.9988 129.72,31.9988 129.948,31.9988 130.175,31.9988 130.403,31.9988 
 130.631,31.9988 130.859,31.9988 131.086,31.9988 131.314,31.9988 131.542,31.9988 131.77,31.9988 131.997,31.9988 132.225,31.9988 132.453,31.9988 132.681,31.9988 
 132.908,31.9988 133.136,31.9988 133.364,31.9988 133.592,31.9988 133.819,31.9988 134.047,31.9988 134.275,31.9988 134.502,31.9988 134.73,31.9988 134.958,31.9988 
 135.186,31.9988 135.413,31.9988 135.641,31.9988 135.869,31.9988 136.097,31.9988 136.324,31.9988 136.552,31.9988 136.78,31.9988 137.008,31.9988 137.235,31.9988 
 137.463,31.9988 137.691,31.9988 137.919,31.9988 138.146,31.9988 138.374,31.9988 138.602,31.9988 138.83,31.9988 139.057,31.9988 139.285,31.9988 139.513,31.9988 
 139.513,31.9988 139.513,31.7711 139.513,31.5433 139.513,31.3156 139.513,31.0878 139.513,30.8601 139.513,30.6323 139.513,30.4046 139.513,30.1769 139.513,29.9491 
 139.513,29.7214 139.513,29.4936 139.513,29.2659 139.513,29.0381 139.513,28.8104 139.513,28.5826 139.513,28.3549 139.513,28.1271 139.513,27.8994 139.513,27.6716 
 139.513,27.4439 139.513,27.2162 139.513,26.9884 139.513,26.7607 139.513,26.5329 139.513,26.3052 139.513,26.0774 139.513,25.8497 139.513,25.6219 139.513,25.3942 
 139.513,25.1664 139.513,24.9387 139.513,24.711 139.513,24.4832 139.513,24.2555 139.513,24.0277 139.513,23.8 139.513,23.5722 139.513,23.3445 139.513,23.1167 
 139.513,22.889 139.513,22.6612 139.513,22.4335 139.513,22.2057 139.513,21.978 139.513,21.7503 139.513,21.5225 139.513,21.2948 139.513,21.067 139.513,20.8393 
 139.513,20.8393 139.285,20.8393 139.057,20.8393 138.83,20.8393 138.602,20.8393 138.374,20.8393 138.146,20.8393 137.919,20.8393 137.691,20.8393 137.463,20.8393 
 137.235,20.8393 137.008,20.8393 136.78,20.8393 136.552,20.8393 136.324,20.8393 136.097,20.8393 135.869,20.8393 135.641,20.8393 135.413,20.8393 135.186,20.8393 
 134.958,20.8393 134.73,20.8393 134.502,20.8393 134.275,20.8393 134.047,20.8393 133.819,20.8393 133.592,20.8393 133.364,20.8393 133.136,20.8393 132.908,20.8393 
 132.681,20.8393 132.453,20.8393 132.225,20.8393 131.997,20.8393 131.77,20.8393 131.542,20.8393 131.314,20.8393 131.086,20.8393 130.859,20.8393 130.631,20.8393 
 130.403,20.8393 130.175,20.8393 129.948,20.8393 129.72,20.8393 129.492,20.8393 129.264,20.8393 129.037,20.8393 128.809,20.8393 128.581,20.8393 128.353,20.8393 
 128.353,20.8393 128.353,21.067 128.353,21.2948 128.353,21.5225 128.353,21.7503 128.353,21.978 128.353,22.2057 128.353,22.4335 128.353,22.6612 128.353,22.889 
 128.353,23.1167 128.353,23.3445 128.353,23.5722 128.353,23.8 128.353,24.0277 128.353,24.2555 128.353,24.4832 128.353,24.711 128.353,24.9387 128.353,25.1664 
 128.353,25.3942 128.353,25.6219 128.353,25.8497 128.353,26.0774 128.353,26.3052 128.353,26.5329 128.353,26.7607 128.353,26.9884 128.353,27.2162 128.353,27.4439 
 128.353,27.6716 128.353,27.8994 128.353,28.1271 128.353,28.3549 128.353,28.5826 128.353,28.8104 128.353,29.0381 128.353,29.2659 128.353,29.4936 128.353,29.7214 
 128.353,29.9491 128.353,30.1769 128.353,30.4046 128.353,30.6323 128.353,30.8601 128.353,31.0878 128.353,31.3156 128.353,31.5433 128.353,31.7711 128.353,31.9988 
 128.353,31.9988 
 " fill="#009af9" fill-opacity="0.5"/>
<polyline clip-path="url(#clip7802)" style="stroke:#000000; stroke-width:1; stroke-opacity:0.5; fill:none" points="
 128.353,31.9988 128.581,31.9988 128.809,31.9988 129.037,31.9988 129.264,31.9988 129.492,31.9988 129.72,31.9988 129.948,31.9988 130.175,31.9988 130.403,31.9988 
 130.631,31.9988 130.859,31.9988 131.086,31.9988 131.314,31.9988 131.542,31.9988 131.77,31.9988 131.997,31.9988 132.225,31.9988 132.453,31.9988 132.681,31.9988 
 132.908,31.9988 133.136,31.9988 133.364,31.99

In [45]:
@manipulate for i in slider(1:length(xsets), value=1)
    p = plot(xlim=(0, 5), ylim=(0, 3), leg=false, aspect_ratio=1)
    for j in 1:i
        plot!(xsets[j], ysets[j], linetype=:shape, alpha=0.5)
    end
    p
end


Interact.Slider{Int64}(49: "input-15" = 1 Int64 , "", 1, 1:61, "horizontal", true, "d", true)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
<polygon clip-path="url(#clip5002)" points="
128.353,31.9988 128.581,31.9988 128.809,31.9988 129.037,31.9988 129.264,31.9988 129.492,31.9988 129.72,31.9988 129.948,31.9988 130.175,31.9988 130.403,31.9988 
 130.631,31.9988 130.859,31.9988 131.086,31.9988 131.314,31.9988 131.542,31.9988 131.77,31.9988 131.997,31.9988 132.225,31.9988 132.453,31.9988 132.681,31.9988 
 132.908,31.9988 133.136,31.9988 133.364,31.9988 133.592,31.9988 133.819,31.9988 134.047,31.9988 134.275,31.9988 134.502,31.9988 134.73,31.9988 134.958,31.9988 
 135.186,31.9988 135.413,31.9988 135.641,31.9988 135.869,31.9988 136.097,31.9988 136.324,31.9988 136.552,31.9988 136.78,31.9988 137.008,31.9988 137.235,31.9988 
 137.463,31.9988 137.691,31.9988 137.919,31.9988 138.146,31.9988 138.374,31.9988 138.602,31.9988 138.83,31.9988 139.057,31.9988 139.285,31.9988 139.513,31.9988 
 139.513,31.9988 139.513,31.7711 139.513,31.5433 139.513,31.3156 139.513,31.0878 139.513,30.8601 139.513,30.6323 139.513,30.4046 139.513,30.1769 139.513,29.9491 
 139.513,29.7214 139.513,29.4936 139.513,29.2659 139.513,29.0381 139.513,28.8104 139.513,28.5826 139.513,28.3549 139.513,28.1271 139.513,27.8994 139.513,27.6716 
 139.513,27.4439 139.513,27.2162 139.513,26.9884 139.513,26.7607 139.513,26.5329 139.513,26.3052 139.513,26.0774 139.513,25.8497 139.513,25.6219 139.513,25.3942 
 139.513,25.1664 139.513,24.9387 139.513,24.711 139.513,24.4832 139.513,24.2555 139.513,24.0277 139.513,23.8 139.513,23.5722 139.513,23.3445 139.513,23.1167 
 139.513,22.889 139.513,22.6612 139.513,22.4335 139.513,22.2057 139.513,21.978 139.513,21.7503 139.513,21.5225 139.513,21.2948 139.513,21.067 139.513,20.8393 
 139.513,20.8393 139.285,20.8393 139.057,20.8393 138.83,20.8393 138.602,20.8393 138.374,20.8393 138.146,20.8393 137.919,20.8393 137.691,20.8393 137.463,20.8393 
 137.235,20.8393 137.008,20.8393 136.78,20.8393 136.552,20.8393 136.324,20.8393 136.097,20.8393 135.869,20.8393 135.641,20.8393 135.413,20.8393 135.186,20.8393 
 134.958,20.8393 134.73,20.8393 134.502,20.8393 134.275,20.8393 134.047,20.8393 133.819,20.8393 133.592,20.8393 133.364,20.8393 133.136,20.8393 132.908,20.8393 
 132.681,20.8393 132.453,20.8393 132.225,20.8393 131.997,20.8393 131.77,20.8393 131.542,20.8393 131.314,20.8393 131.086,20.8393 130.859,20.8393 130.631,20.8393 
 130.403,20.8393 130.175,20.8393 129.948,20.8393 129.72,20.8393 129.492,20.8393 129.264,20.8393 129.037,20.8393 128.809,20.8393 128.581,20.8393 128.353,20.8393 
 128.353,20.8393 128.353,21.067 128.353,21.2948 128.353,21.5225 128.353,21.7503 128.353,21.978 128.353,22.2057 128.353,22.4335 128.353,22.6612 128.353,22.889 
 128.353,23.1167 128.353,23.3445 128.353,23.5722 128.353,23.8 128.353,24.0277 128.353,24.2555 128.353,24.4832 128.353,24.711 128.353,24.9387 128.353,25.1664 
 128.353,25.3942 128.353,25.6219 128.353,25.8497 128.353,26.0774 128.353,26.3052 128.353,26.5329 128.353,26.7607 128.353,26.9884 128.353,27.2162 128.353,27.4439 
 128.353,27.6716 128.353,27.8994 128.353,28.1271 128.353,28.3549 128.353,28.5826 128.353,28.8104 128.353,29.0381 128.353,29.2659 128.353,29.4936 128.353,29.7214 
 128.353,29.9491 128.353,30.1769 128.353,30.4046 128.353,30.6323 128.353,30.8601 128.353,31.0878 128.353,31.3156 128.353,31.5433 128.353,31.7711 128.353,31.9988 
 128.353,31.9988 
 " fill="#009af9" fill-opacity="0.5"/>
<polyline clip-path="url(#clip5002)" style="stroke:#000000; stroke-width:1; stroke-opacity:0.5; fill:none" points="
 128.353,31.9988 128.581,31.9988 128.809,31.9988 129.037,31.9988 129.264,31.9988 129.492,31.9988 129.72,31.9988 129.948,31.9988 130.175,31.9988 130.403,31.9988 
 130.631,31.9988 130.859,31.9988 131.086,31.9988 131.314,31.9988 131.542,31.9988 131.77,31.9988 131.997,31.9988 132.225,31.9988 132.453,31.9988 132.681,31.9988 
 132.908,31.9988 133.136,31.9988 133.364,31.99

In [20]:
t = Taylor1(10)

 1.0 t + 𝒪(t¹¹)

In [21]:
?Taylor1

search: Taylor1 TaylorN taylor_var TaylorModel TaylorModels TaylorSeries



```
Taylor1{T<:Number} <: AbstractSeries{T}
```

DataType for polynomial expansions in one independent variable.

**Fields:**

  * `coeffs :: Array{T,1}` Expansion coefficients; the $i$-th   component is the coefficient of degree $i-1$ of the expansion.
  * `order  :: Int64` Maximum order (degree) of the polynomial.

Note that `Taylor1` variables are callable. For more information, see [`evaluate`](@ref).

```
Taylor1([T::Type=Float64], [order::Int=1])
```

Shortcut to define the independent variable of a `Taylor1{T}` polynomial of given `order`. The default type for `T` is `Float64`.

```julia
julia> Taylor1(16)
 1.0 t + 𝒪(t¹⁷)

julia> Taylor1(Rational{Int}, 4)
 1//1 t + 𝒪(t⁵)
```


In [ ]:
TaylorModel(3, 1..1, 0.5..1.5)